In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# Training settings
batch_size=64
test_batch_size=1000
epochs=1
lr=0.01
momentum=0.5
no_cuda=False,
seed=1 #help='random seed (default: 1)')
log_interval=10 #how many batches to wait before logging training status

cuda = not no_cuda and torch.cuda.is_available()
print(cuda)
cuda = torch.cuda.is_available()
print(cuda)

False
False


In [3]:
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

Params for if using cuda

In [4]:
params = {'num_workers': 1, 'pin_memory': True} if cuda else {}


#### Loading MNIST data and normalizing it

In [5]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **params)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **params)

#### Creating the model

nn.Module = Base class for all neural network modules.

2d conv = nn.Conv2d(filtersize_input,filters_out, kernel_size)   #padding??  
fully connected = nn.Linear(inputs,outputs) # ??? why Linear Applies a linear transformation to the incoming data: y=Ax+b  
dropout = nn.Dropout(p=0.5, inplace=False)  

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout(0.5)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

model = Net()

In [7]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [8]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [9]:
def test(epoch):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))



In [10]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.340125
Train Epoch: 1 [640/60000 (1%)]	Loss: 2.318111
Train Epoch: 1 [1280/60000 (2%)]	Loss: 2.264411
Train Epoch: 1 [1920/60000 (3%)]	Loss: 2.206053
Train Epoch: 1 [2560/60000 (4%)]	Loss: 2.166254
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.188091
Train Epoch: 1 [3840/60000 (6%)]	Loss: 2.065116
Train Epoch: 1 [4480/60000 (7%)]	Loss: 1.819609
Train Epoch: 1 [5120/60000 (9%)]	Loss: 1.730206
Train Epoch: 1 [5760/60000 (10%)]	Loss: 1.522603
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.485478
Train Epoch: 1 [7040/60000 (12%)]	Loss: 1.224622
Train Epoch: 1 [7680/60000 (13%)]	Loss: 1.022467
Train Epoch: 1 [8320/60000 (14%)]	Loss: 1.098657
Train Epoch: 1 [8960/60000 (15%)]	Loss: 1.113300
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.107109
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.981816
Train Epoch: 1 [10880/60000 (18%)]	Loss: 0.972617
Train Epoch: 1 [11520/60000 (19%)]	Loss: 0.871269
Train Epoch: 1 [12160/60000 (20%)]	Loss: 1.003348
Train Epoch: 1 [12800/60000 (